# 🤖 Configurable LLM Chatbot with Fine-Tuning

This notebook provides a complete RAG (Retrieval-Augmented Generation) chatbot with fine-tuning capabilities for custom models.

## Features:
- 📚 Document loading (PDF, TXT, MD)
- 🔍 Vector search with ChromaDB
- 🎯 Custom fine-tuning of language models
- 💬 Interactive Q&A interface
- 🔄 Domain switching

## Setup Instructions:
1. Run all cells in order
2. Upload your documents to the `Documents/` folder
3. Configure your domain and fine-tune if needed
4. Start chatting!

## 🛠️ Environment Setup

In [24]:
# Install required dependencies
!pip install -q langchain langchain-core langchain-community langchain-text-splitters
!pip install -q langchain-ollama langchain-chroma langchain-huggingface
!pip install -q chromadb sentence-transformers pypdf pandas
!pip install -q transformers datasets torch accelerate bitsandbytes
!pip install -q peft trl wandb

print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


In [25]:
# Core imports
import os
import shutil
import json
import torch
import warnings
from pathlib import Path
from typing import List, Dict, Optional, Tuple

# LangChain imports
from langchain_ollama import OllamaLLM 
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains import RetrievalQA
from langchain_core.documents import Document

# Fine-tuning imports
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    BitsAndBytesConfig, DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import wandb

warnings.filterwarnings('ignore')
print("✅ All imports completed!")

✅ All imports completed!


## ⚙️ Configuration

In [26]:
# Configuration class
class Config:
    # Model settings
    BASE_MODEL = "microsoft/DialoGPT-medium"  # Base model for fine-tuning
    OLLAMA_MODEL = "chatter"  # Ollama model for RAG
    
    # Fine-tuning settings
    USE_LORA = True
    LORA_R = 16
    LORA_ALPHA = 32
    LORA_DROPOUT = 0.1
    
    # Training settings
    BATCH_SIZE = 4
    GRAD_ACCUM_STEPS = 4
    LEARNING_RATE = 2e-4
    NUM_EPOCHS = 3
    MAX_LENGTH = 512
    
    # RAG settings
    EMBEDDING_MODEL = "all-MiniLM-L6-v2"
    CHUNK_SIZE = 1000
    CHUNK_OVERLAP = 200
    RETRIEVAL_K = 4
    
    # Paths
    DOCS_FOLDER = "./Documents"
    FINE_TUNED_MODEL_PATH = "./fine_tuned_model"
    CHROMA_DB_PATH = "./chroma_db"
    
# Initialize config
config = Config()
print(f"📋 Configuration loaded:")
print(f"  Base Model: {config.BASE_MODEL}")
print(f"  Ollama Model: {config.OLLAMA_MODEL}")
print(f"  Use LoRA: {config.USE_LORA}")
print(f"  Batch Size: {config.BATCH_SIZE}")
print(f"  Learning Rate: {config.LEARNING_RATE}")

📋 Configuration loaded:
  Base Model: microsoft/DialoGPT-medium
  Ollama Model: chatter
  Use LoRA: True
  Batch Size: 4
  Learning Rate: 0.0002


## 🎯 Fine-Tuning Component

In [27]:
class ModelFineTuner:
    """Fine-tuning component for custom language models"""
    
    def __init__(self, config: Config):
        self.config = config
        self.tokenizer = None
        self.model = None
        self.peft_model = None
        
    def setup_model(self):
        """Setup model and tokenizer for fine-tuning"""
        print("🔧 Setting up model for fine-tuning...")
        
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.config.BASE_MODEL,
            padding_side="right",
            truncation=True
        )
        
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Quantization config (for memory efficiency)
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=False,
        )
        
        # Load model
        self.model = AutoModelForCausalLM.from_pretrained(
            self.config.BASE_MODEL,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
        
        # Setup LoRA if enabled
        if self.config.USE_LORA:
            self._setup_lora()
            
        print("✅ Model setup completed!")
        
    def _setup_lora(self):
        """Setup LoRA for parameter-efficient fine-tuning"""
        lora_config = LoraConfig(
            r=self.config.LORA_R,
            lora_alpha=self.config.LORA_ALPHA,
            target_modules=["c_attn", "c_proj"],
            lora_dropout=self.config.LORA_DROPOUT,
            bias="none",
            task_type=TaskType.CAUSAL_LM
        )
        
        self.peft_model = get_peft_model(self.model, lora_config)
        self.peft_model.print_trainable_parameters()
        print("✅ LoRA setup completed!")
        
    def prepare_dataset(self, conversations: List[Dict]) -> Dataset:
        """Prepare dataset for fine-tuning"""
        print("📊 Preparing dataset...")
        
        def format_conversation(conv):
            """Format conversation for training"""
            if isinstance(conv, dict) and 'messages' in conv:
                # Chat format
                formatted = ""
                for msg in conv['messages']:
                    role = msg.get('role', 'user')
                    content = msg.get('content', '')
                    formatted += f"{role}: {content}\n"
                return formatted + "assistant: "
            else:
                # Simple Q&A format
                return f"user: {conv.get('question', '')}\nassistant: {conv.get('answer', '')}"
        
        # Format and tokenize
        formatted_texts = [format_conversation(conv) for conv in conversations]
        
        dataset = Dataset.from_dict({"text": formatted_texts})
        
        def tokenize_function(examples):
            return self.tokenizer(
                examples["text"],
                truncation=True,
                padding=True,
                max_length=self.config.MAX_LENGTH,
                return_tensors="pt"
            )
        
        tokenized_dataset = dataset.map(tokenize_function, batched=True)
        print(f"✅ Dataset prepared with {len(tokenized_dataset)} examples")
        
        return tokenized_dataset
        
    def fine_tune(self, dataset: Dataset, save_path: str = None):
        """Fine-tune the model"""
        print("🚀 Starting fine-tuning...")
        
        if save_path is None:
            save_path = self.config.FINE_TUNED_MODEL_PATH
            
        # Training arguments
        training_args = TrainingArguments(
            output_dir=save_path,
            num_train_epochs=self.config.NUM_EPOCHS,
            per_device_train_batch_size=self.config.BATCH_SIZE,
            gradient_accumulation_steps=self.config.GRAD_ACCUM_STEPS,
            learning_rate=self.config.LEARNING_RATE,
            fp16=True,
            logging_steps=10,
            save_steps=100,
            save_total_limit=2,
            evaluation_strategy="no",
            load_best_model_at_end=False,
            report_to="none",  # Disable wandb logging unless needed
        )
        
        # Data collator
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False,
        )
        
        # Trainer
        model_to_train = self.peft_model if self.peft_model else self.model
        
        trainer = Trainer(
            model=model_to_train,
            args=training_args,
            train_dataset=dataset,
            data_collator=data_collator,
        )
        
        # Start training
        trainer.train()
        
        # Save model
        trainer.save_model(save_path)
        self.tokenizer.save_pretrained(save_path)
        
        print(f"✅ Fine-tuning completed! Model saved to {save_path}")
        
    def load_fine_tuned_model(self, model_path: str):
        """Load a fine-tuned model"""
        print(f"📂 Loading fine-tuned model from {model_path}...")
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        
        if self.config.USE_LORA:
            self.model = AutoModelForCausalLM.from_pretrained(
                self.config.BASE_MODEL,
                device_map="auto",
                torch_dtype=torch.float16
            )
            self.peft_model = get_peft_model(self.model, LoraConfig(
                r=self.config.LORA_R,
                lora_alpha=self.config.LORA_ALPHA,
                target_modules=["c_attn", "c_proj"],
                lora_dropout=self.config.LORA_DROPOUT,
                bias="none",
                task_type=TaskType.CAUSAL_LM
            ))
            
            # Load LoRA weights
            from peft import PeftModel
            self.peft_model = PeftModel.from_pretrained(self.model, model_path)
        else:
            self.model = AutoModelForCausalLM.from_pretrained(model_path)
            
        print("✅ Fine-tuned model loaded successfully!")
        
    def generate_response(self, prompt: str, max_length: int = 100) -> str:
        """Generate response using fine-tuned model"""
        if not self.tokenizer or not (self.model or self.peft_model):
            return "Model not loaded. Please setup or load a model first."
            
        model_to_use = self.peft_model if self.peft_model else self.model
        
        # Tokenize input
        inputs = self.tokenizer.encode(prompt, return_tensors="pt", truncation=True).to(model_to_use.device)
        
        # Generate response
        with torch.no_grad():
            outputs = model_to_use.generate(
                inputs,
                max_length=inputs.shape[1] + max_length,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        # Decode response
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Remove the original prompt
        if response.startswith(prompt):
            response = response[len(prompt):].strip()
            
        return response

print("✅ Fine-tuning component defined!")

✅ Fine-tuning component defined!


## 📚 RAG Component

In [28]:
class ConfigurableDocBot:
    """RAG-based document chatbot"""
    
    def __init__(self, config: Config, model_name: str = None):
        """Initialize the configurable RAG chatbot"""
        self.config = config
        self.model_name = model_name or config.OLLAMA_MODEL
        
        # Initialize components
        self.llm = None
        self.vectorstore = None
        self.qa_chain = None
        self.current_domain = None
        
        print("🚀 Initializing Configurable Document Bot...")
        self._setup_embeddings()
        self._setup_llm()
        
    def _setup_embeddings(self):
        """Setup embedding model"""
        print("⏳ Loading embeddings model (first time downloads ~90MB)...")
        
        self.embeddings = HuggingFaceEmbeddings(
            model_name=self.config.EMBEDDING_MODEL,
            model_kwargs={'device': 'cpu'}
        )
        print("✅ Embeddings model loaded!\n")
        
    def _setup_llm(self):
        """Setup language model"""
        try:
            self.llm = OllamaLLM(model=self.model_name)
            print(f"✅ Ollama LLM '{self.model_name}' loaded!")
        except Exception as e:
            print(f"⚠️ Could not load Ollama LLM: {e}")
            print("💡 Make sure Ollama is installed and the model is available")
            
    def load_domain(self, domain_name: str, docs_folder: str) -> bool:
        """Load documents for a specific domain/customer"""
        print(f"\n{'='*60}")
        print(f"📂 Loading domain: {domain_name}")
        print(f"📁 Document folder: {docs_folder}")
        print(f"{'='*60}")
        
        self.current_domain = domain_name
        
        # Clean up old vector store
        db_path = f"{self.config.CHROMA_DB_PATH}_{domain_name}"
        if os.path.exists(db_path):
            print(f"🗑️ Removing old vector database...")
            shutil.rmtree(db_path)
        
        # Load documents
        documents = self._load_documents(docs_folder)
        
        if not documents:
            print(f"\n⚠️ WARNING: No documents found in {docs_folder}")
            print(f"ℹ️ The bot will respond that information is not available")
            self.vectorstore = None
            self.qa_chain = None
            return False
        
        # Process documents and create vector database
        self._process_documents(documents, domain_name)
        
        # Setup QA chain
        self._setup_qa_chain()
        
        print(f"\n✅ Domain '{domain_name}' configured successfully!")
        print(f"📊 Ready to answer questions about {len(documents)} documents\n")
        return True
        
    def _load_documents(self, docs_folder: str) -> List[Document]:
        """Load all supported document types from folder"""
        documents = []
        folder_path = Path(docs_folder)
        
        if not folder_path.exists():
            print(f"❌ Error: Folder '{docs_folder}' does not exist")
            return documents
        
        print("\n📄 Loading documents...")
        
        # Load markdown and text files
        for ext in ['*.md', '*.txt']:
            for file_path in folder_path.rglob(ext):
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                        if content.strip():  # Only if not empty
                            doc = Document(
                                page_content=content,
                                metadata={"source": str(file_path)}
                            )
                            documents.append(doc)
                            print(f"  ✓ Loaded {file_path.name}")
                except Exception as e:
                    print(f"  ⚠️ Error loading {file_path.name}: {e}")
        
        # Load PDF files
        try:
            from pypdf import PdfReader
            for file_path in folder_path.rglob('*.pdf'):
                try:
                    reader = PdfReader(str(file_path))
                    text = ""
                    for page in reader.pages:
                        text += page.extract_text() + "\n"
                    
                    if text.strip():
                        doc = Document(
                            page_content=text,
                            metadata={"source": str(file_path)}
                        )
                        documents.append(doc)
                        print(f"  ✓ Loaded {file_path.name}")
                except Exception as e:
                    print(f"  ⚠️ Error loading {file_path.name}: {e}")
        except ImportError:
            print("  ℹ️ pypdf not available, skipping PDF files")
        
        if documents:
            print(f"\n📚 Total documents loaded: {len(documents)}")
        
        return documents
        
    def _process_documents(self, documents: List[Document], domain_name: str):
        """Split documents into chunks and create vector database"""
        print("\n⚙️ Processing documents...")
        
        # Split documents into smaller chunks for better retrieval
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.config.CHUNK_SIZE,
            chunk_overlap=self.config.CHUNK_OVERLAP,
            length_function=len,
        )
        
        texts = text_splitter.split_documents(documents)
        print(f"  ✓ Split into {len(texts)} chunks")
        
        # Create vector database
        print(f"  ⏳ Creating vector database...")
        self.vectorstore = Chroma.from_documents(
            documents=texts,
            embedding=self.embeddings,
            persist_directory=f"{self.config.CHROMA_DB_PATH}_{domain_name}"
        )
        print(f"  ✓ Vector database created and persisted")
        
    def _setup_qa_chain(self):
        """Setup the question-answering chain"""
        if self.vectorstore is None:
            print("⚠️ No vector store - QA chain not created")
            return
        
        if self.llm is None:
            print("⚠️ No LLM available - QA chain not created")
            return
            
        print("  ⏳ Setting up QA chain...")
        
        # Create retriever (finds relevant chunks)
        retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": self.config.RETRIEVAL_K}
        )
        
        # Create prompt template
        prompt_template = """CONTEXT FROM DOCUMENTS:
{context}

QUESTION: {question}

INSTRUCTIONS:
1. Answer using ONLY the information in the CONTEXT above
2. Follow a step-by-step format from your system prompt
3. If the answer is not in the CONTEXT, respond: "This information is not available to me. If you want, I can contact a human agent!"
4. Cite specific sections or sources when possible

YOUR ANSWER:"""

        PROMPT = PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"]
        )
        
        # Create the QA chain
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": PROMPT}
        )
        
        print("  ✓ QA chain ready")
        
    def ask(self, question: str) -> str:
        """Ask a question about the loaded documents"""
        if not self.current_domain:
            return "❌ No domain loaded. Use load_domain() first."
        
        if self.qa_chain is None:
            return "This information is not available to me. If you want, I can contact a human agent!"
        
        print(f"\n{'─'*60}")
        print(f"🔍 Question: {question}")
        print(f"📂 Domain: {self.current_domain}")
        print("⏳ Searching documents...")
        
        try:
            # Query the chain
            result = self.qa_chain.invoke({"query": question})
            
            answer = result["result"]
            
            # Display source documents
            if result.get("source_documents"):
                print(f"\n📄 Sources used:")
                for i, doc in enumerate(result["source_documents"][:3], 1):
                    source = doc.metadata.get("source", "Unknown")
                    snippet = doc.page_content[:150].replace("\n", " ")
                    print(f"  {i}. {os.path.basename(source)}")
                    print(f"     Preview: {snippet}...")
            
            print(f"{'─'*60}\n")
            return answer
            
        except Exception as e:
            print(f"\n❌ Error: {str(e)}\n")
            return f"Error processing question: {str(e)}"
        
    def switch_domain(self, domain_name: str, docs_folder: str) -> bool:
        """Switch to a different domain/customer"""
        print(f"\n🔄 Switching from '{self.current_domain}' to '{domain_name}'...")
        return self.load_domain(domain_name, docs_folder)

print("✅ RAG component defined!")

✅ RAG component defined!


## 📝 Sample Data Setup

In [29]:
# Create sample documents for testing
def create_sample_documents():
    """Create sample documents for demonstration"""
    
    # Create Documents directory
    docs_dir = Path(config.DOCS_FOLDER)
    docs_dir.mkdir(exist_ok=True)
    
    # Create subdirectories
    medical_dir = docs_dir / "Medical"
    legal_dir = docs_dir / "Legal"
    tech_dir = docs_dir / "Tech"
    
    for subdir in [medical_dir, legal_dir, tech_dir]:
        subdir.mkdir(exist_ok=True)
    
    # Medical documents
    medical_content = """
# Medical Guidelines

## Common Medical Conditions

### Hypertension
Hypertension, or high blood pressure, is a common medical condition affecting millions worldwide. 
Normal blood pressure is typically around 120/80 mmHg. Hypertension is diagnosed when blood 
pressure consistently exceeds 130/80 mmHg.

### Diabetes
Diabetes mellitus is a group of metabolic disorders characterized by high blood sugar levels 
over a prolonged period. Type 2 diabetes is the most common form, accounting for 90-95% of cases.

## Treatment Recommendations

For hypertension patients, lifestyle modifications including:
- Reduced sodium intake
- Regular physical activity
- Weight management
- Stress reduction techniques

For diabetes management:
- Regular blood glucose monitoring
- Dietary modifications
- Physical activity
- Medication as prescribed
"""
    
    with open(medical_dir / "guidelines.md", "w") as f:
        f.write(medical_content)
    
    # Legal documents
    legal_content = """
# Legal Information

## Contract Law Basics

A contract is a legally binding agreement between two or more parties. For a contract to be 
valid, it must contain several essential elements:

1. **Offer**: One party proposes to enter into an agreement
2. **Acceptance**: The other party agrees to the terms
3. **Consideration**: Something of value is exchanged
4. **Capacity**: All parties have the legal ability to contract
5. **Legality**: The contract's purpose is legal

## Types of Contracts

### Employment Contracts
Employment contracts outline the terms of employment between an employer and employee. 
Key provisions include salary, work hours, responsibilities, and termination conditions.

### Service Agreements
Service agreements define the terms under which one party provides services to another. 
These should clearly specify scope of work, payment terms, and deliverables.
"""
    
    with open(legal_dir / "contracts.md", "w") as f:
        f.write(legal_content)
    
    # Tech documents
    tech_content = """
# Technical Documentation

## Python Programming

Python is a high-level, interpreted programming language known for its simplicity and 
readability. It's widely used in web development, data science, machine learning, and automation.

### Key Features
- Dynamic typing
- Automatic memory management
- Extensive standard library
- Cross-platform compatibility

## Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and 
improve from experience without being explicitly programmed.

### Common Algorithms

#### Linear Regression
Linear regression is used for predicting continuous values. It models the relationship 
between dependent and independent variables using a linear approach.

#### Decision Trees
Decision trees are non-parametric supervised learning methods used for classification and 
regression. They work by learning simple decision rules inferred from data features.

### Best Practices
1. Data preprocessing and cleaning
2. Feature engineering and selection
3. Model validation using cross-validation
4. Hyperparameter tuning
5. Regular monitoring and retraining
"""
    
    with open(tech_dir / "programming.md", "w") as f:
        f.write(tech_content)
    
    print(f"✅ Sample documents created in {config.DOCS_FOLDER}")

# Create sample documents
create_sample_documents()

✅ Sample documents created in ./Documents


In [30]:
# Create sample training data for fine-tuning
def create_sample_training_data():
    """Create sample conversation data for fine-tuning"""
    
    training_data = [
        {
            "question": "What is hypertension?",
            "answer": "Hypertension, or high blood pressure, is a common medical condition where blood pressure consistently exceeds 130/80 mmHg. It affects millions worldwide and requires lifestyle modifications for management."
        },
        {
            "question": "How is diabetes managed?",
            "answer": "Diabetes management involves regular blood glucose monitoring, dietary modifications, physical activity, and medication as prescribed by healthcare providers. Type 2 diabetes is the most common form."
        },
        {
            "question": "What makes a contract valid?",
            "answer": "A valid contract requires five essential elements: offer, acceptance, consideration, capacity, and legality. All parties must have the legal ability to contract, and the contract's purpose must be legal."
        },
        {
            "question": "What is Python used for?",
            "answer": "Python is widely used in web development, data science, machine learning, and automation. It's known for simplicity, readability, dynamic typing, and extensive standard library support."
        },
        {
            "question": "What are common machine learning algorithms?",
            "answer": "Common machine learning algorithms include linear regression for predicting continuous values, decision trees for classification and regression, and various others like neural networks and support vector machines."
        }
    ]
    
    return training_data

# Create training data
sample_training_data = create_sample_training_data()
print(f"✅ Created {len(sample_training_data)} training examples")
print("Sample training data:")
for i, example in enumerate(sample_training_data[:2]):
    print(f"\n{i+1}. Q: {example['question']}")
    print(f"   A: {example['answer'][:100]}...")

✅ Created 5 training examples
Sample training data:

1. Q: What is hypertension?
   A: Hypertension, or high blood pressure, is a common medical condition where blood pressure consistentl...

2. Q: How is diabetes managed?
   A: Diabetes management involves regular blood glucose monitoring, dietary modifications, physical activ...


## 🚀 Demo: Fine-Tuning + RAG

In [31]:
# Initialize both components
print("🔧 Initializing components...")

# Initialize fine-tuner
fine_tuner = ModelFineTuner(config)

# Initialize RAG bot
rag_bot = ConfigurableDocBot(config)

print("✅ Components initialized!")

🔧 Initializing components...
🚀 Initializing Configurable Document Bot...
⏳ Loading embeddings model (first time downloads ~90MB)...
✅ Embeddings model loaded!

✅ Ollama LLM 'chatter' loaded!
✅ Components initialized!


### Step 1: Fine-Tuning Demo (Optional)

In [32]:
# Uncomment to run fine-tuning (this may take a while)
# Note: This is optional and requires significant computational resources

print("🎯 Starting fine-tuning demo...")
print("⚠️ This may take 10-30 minutes depending on your GPU...")

# Setup model for fine-tuning
fine_tuner.setup_model()

# Prepare dataset
dataset = fine_tuner.prepare_dataset(sample_training_data)

# Run fine-tuning (commented out by default to save time)
# fine_tuner.fine_tune(dataset, "./demo_fine_tuned_model")

print("\n💡 Fine-tuning setup complete!")
print("   Uncomment the fine_tuner.fine_tune() line to actually run training.")

🎯 Starting fine-tuning demo...
⚠️ This may take 10-30 minutes depending on your GPU...
🔧 Setting up model for fine-tuning...
trainable params: 4,325,376 || all params: 359,148,544 || trainable%: 1.2043
✅ LoRA setup completed!
✅ Model setup completed!
📊 Preparing dataset...


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

✅ Dataset prepared with 5 examples

💡 Fine-tuning setup complete!
   Uncomment the fine_tuner.fine_tune() line to actually run training.


### Step 2: RAG Demo

In [33]:
# Load a domain for RAG
print("📚 Loading RAG domain...")

# Load medical domain
success = rag_bot.load_domain("medical", f"{config.DOCS_FOLDER}/Medical")

if success:
    print("✅ Medical domain loaded successfully!")
else:
    print("⚠️ Failed to load medical domain")

📚 Loading RAG domain...

📂 Loading domain: medical
📁 Document folder: ./Documents/Medical

📄 Loading documents...
  ✓ Loaded guidelines.md

📚 Total documents loaded: 1

⚙️ Processing documents...
  ✓ Split into 1 chunks
  ⏳ Creating vector database...


InternalError: Query error: Database error: error returned from database: (code: 1032) attempt to write a readonly database

In [ ]:
# Test RAG with sample questions
test_questions = [
    "What is hypertension?",
    "How is diabetes managed?",
    "What are the treatment recommendations for high blood pressure?"
]

print("🔍 Testing RAG with sample questions...\n")

for question in test_questions:
    answer = rag_bot.ask(question)
    print(f"Q: {question}")
    print(f"A: {answer}\n")
    print("-" * 80)

🔍 Testing RAG with sample questions...


────────────────────────────────────────────────────────────
🔍 Question: What is hypertension?
📂 Domain: medical
⏳ Searching documents...

❌ Error: [Errno 111] Connection refused

Q: What is hypertension?
A: Error processing question: [Errno 111] Connection refused

--------------------------------------------------------------------------------

────────────────────────────────────────────────────────────
🔍 Question: How is diabetes managed?
📂 Domain: medical
⏳ Searching documents...

❌ Error: [Errno 111] Connection refused

Q: How is diabetes managed?
A: Error processing question: [Errno 111] Connection refused

--------------------------------------------------------------------------------

────────────────────────────────────────────────────────────
🔍 Question: What are the treatment recommendations for high blood pressure?
📂 Domain: medical
⏳ Searching documents...

❌ Error: [Errno 111] Connection refused

Q: What are the treatment recomme

### Step 3: Interactive Chat

In [ ]:
# Interactive chat demo
def interactive_chat():
    """Run interactive chat with the RAG bot"""
    
    print("\n" + "="*60)
    print("  💬 INTERACTIVE CHAT DEMO")
    print("="*60)
    print("Commands:")
    print("  • Type your question to ask")
    print("  • 'switch <domain>' - change domain (medical/legal/tech)")
    print("  • 'quit' - exit chat")
    print()
    
    while True:
        try:
            user_input = input(f"[{rag_bot.current_domain}] You: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("\n👋 Goodbye!\n")
                break
            
            if user_input.lower().startswith('switch'):
                parts = user_input.split()
                if len(parts) >= 2:
                    new_domain = parts[1]
                    docs_path = f"{config.DOCS_FOLDER}/{new_domain.capitalize()}"
                    
                    if rag_bot.switch_domain(new_domain, docs_path):
                        print(f"✅ Switched to {new_domain} domain")
                    else:
                        print(f"❌ Failed to switch to {new_domain} domain")
                else:
                    print("Usage: switch <domain> (medical/legal/tech)")
                continue
            
            if not user_input:
                continue
            
            # Ask the question
            answer = rag_bot.ask(user_input)
            print(f"\n🤖 Assistant:\n{answer}\n")
            
        except KeyboardInterrupt:
            print("\n\n👋 Goodbye!\n")
            break
        except Exception as e:
            print(f"\n❌ Error: {str(e)}\n")

# Run interactive chat (uncomment to use)
# interactive_chat()

print("💡 Uncomment interactive_chat() call to start interactive mode")
print("   Or run individual questions using: rag_bot.ask('your question')")

💡 Uncomment interactive_chat() call to start interactive mode
   Or run individual questions using: rag_bot.ask('your question')


## 🔧 Advanced Features

In [ ]:
# Demonstrate domain switching
print("🔄 Demonstrating domain switching...\n")

# Test different domains
domains = [
    ("medical", "What is hypertension?"),
    ("legal", "What makes a contract valid?"),
    ("tech", "What is Python used for?")
]

for domain_name, question in domains:
    print(f"\n--- Switching to {domain_name.upper()} domain ---")
    
    docs_path = f"{config.DOCS_FOLDER}/{domain_name.capitalize()}"
    
    if rag_bot.switch_domain(domain_name, docs_path):
        answer = rag_bot.ask(question)
        print(f"Answer: {answer[:200]}...")
    else:
        print(f"Failed to load {domain_name} domain")
    
    print("\n" + "="*60)

🔄 Demonstrating domain switching...


--- Switching to MEDICAL domain ---

🔄 Switching from 'medical' to 'medical'...

📂 Loading domain: medical
📁 Document folder: ./Documents/Medical
🗑️ Removing old vector database...

📄 Loading documents...
  ✓ Loaded guidelines.md

📚 Total documents loaded: 1

⚙️ Processing documents...
  ✓ Split into 1 chunks
  ⏳ Creating vector database...


InternalError: Query error: Database error: error returned from database: (code: 1032) attempt to write a readonly database

In [ ]:
# Custom prompt engineering
def create_custom_prompt_template(domain_type: str) -> str:
    """Create domain-specific prompt templates"""
    
    templates = {
        "medical": """
CONTEXT FROM MEDICAL DOCUMENTS:
{context}

MEDICAL QUESTION: {question}

INSTRUCTIONS:
1. Provide accurate medical information based on the context
2. Include relevant medical terminology
3. Suggest consulting healthcare professionals for personal medical advice
4. If information is not available, recommend consulting a medical professional

MEDICAL ANSWER:
""",
        
        "legal": """
CONTEXT FROM LEGAL DOCUMENTS:
{context}

LEGAL QUESTION: {question}

INSTRUCTIONS:
1. Provide legal information based on the context
2. Use precise legal terminology
3. Include relevant legal principles and precedents if mentioned
4. Always recommend consulting qualified legal counsel for specific legal advice

LEGAL ANSWER:
""",
        
        "tech": """
CONTEXT FROM TECHNICAL DOCUMENTS:
{context}

TECHNICAL QUESTION: {question}

INSTRUCTIONS:
1. Provide technical information based on the context
2. Include relevant technical details and code examples if available
3. Explain technical concepts clearly
4. Suggest additional resources or documentation when helpful

TECHNICAL ANSWER:
"""
    }
    
    return templates.get(domain_type, templates["tech"])

# Demonstrate custom prompts
print("🎨 Demonstrating custom prompt templates...\n")

for domain in ["medical", "legal", "tech"]:
    template = create_custom_prompt_template(domain)
    print(f"{domain.upper()} Prompt Template:")
    print(template[:300] + "...\n")
    print("-" * 60)

## 📦 Export and Deployment

In [ ]:
# Save components for later use
def save_components():
    """Save trained components and configurations"""
    
    # Create export directory
    export_dir = Path("./exported_components")
    export_dir.mkdir(exist_ok=True)
    
    # Save configuration
    config_dict = {
        "BASE_MODEL": config.BASE_MODEL,
        "OLLAMA_MODEL": config.OLLAMA_MODEL,
        "EMBEDDING_MODEL": config.EMBEDDING_MODEL,
        "CHUNK_SIZE": config.CHUNK_SIZE,
        "CHUNK_OVERLAP": config.CHUNK_OVERLAP,
        "RETRIEVAL_K": config.RETRIEVAL_K,
        "USE_LORA": config.USE_LORA,
        "LORA_R": config.LORA_R,
        "LORA_ALPHA": config.LORA_ALPHA,
    }
    
    with open(export_dir / "config.json", "w") as f:
        json.dump(config_dict, f, indent=2)
    
    # Save sample training data
    with open(export_dir / "training_data.json", "w") as f:
        json.dump(sample_training_data, f, indent=2)
    
    print(f"✅ Components saved to {export_dir}")
    print(f"   - Configuration: config.json")
    print(f"   - Training data: training_data.json")
    print(f"   - Vector databases: {config.CHROMA_DB_PATH}_*")

# Save components
save_components()

## 📋 Summary

This notebook provides a complete fine-tuning and RAG system with:

### ✅ Features Implemented:
- **Fine-tuning Component**: Custom model training with LoRA support
- **RAG Component**: Document retrieval and question answering
- **Domain Switching**: Multiple document domains (Medical, Legal, Tech)
- **Interactive Chat**: Real-time Q&A interface
- **Sample Data**: Pre-configured documents and training data

### 🎯 Use Cases:
1. **Customer Support Bots**: Fine-tune on company-specific conversations
2. **Domain Experts**: Load specialized documents for professional Q&A
3. **Educational Tools**: Create tutoring systems for specific subjects
4. **Research Assistants**: Load academic papers for research Q&A

### 🚀 Next Steps:
1. **Upload Your Documents**: Replace sample docs with your own files
2. **Prepare Training Data**: Create domain-specific conversation data
3. **Run Fine-Tuning**: Train custom models for your use case
4. **Deploy**: Export and deploy to production environments

### 💡 Tips:
- Use GPU runtime for faster fine-tuning
- Start with small datasets for testing
- Monitor training loss to avoid overfitting
- Experiment with different chunk sizes for better retrieval

---

**🎉 Congratulations!** You now have a fully configurable fine-tuning and RAG system ready for deployment!